## Groth16


In [6]:

from scipy.interpolate import lagrange
import numpy as np
import random
import functools 
from numpy.polynomial import polynomial as P
from py_ecc.optimized_bn128 import curve_order, G1, G2, add, multiply, pairing, neg, normalize
import galois

# P = 71
# P = 21888242871839275222246405745257275088548364400416034343698204186575808495617
P = curve_order
GF = galois.GF(P)

np.set_printoptions(precision=2)


L_F = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R_F = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O_F = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [GF(P-3),1,1,2,0,GF(P-1)]])

L_galois = GF(L_F)
R_galois = GF(R_F)
O_galois = GF(O_F)

# pick random values for x and y
x = GF(1)
y = GF(2)


# this is our original formula
v1 = 3*x*x
v2 = v1 * y
out = 3*x*x*y + 5*x*y + GF(P-1)*x + GF(P-2)*y + GF(3)
# the witness vector with the intermediate variables inside
w = GF(np.array([1, out, x, y, v1, v2]))

print("witness {}".format(w))

result = O_galois.dot(w) == np.multiply(L_galois.dot(w),R_galois.dot(w))
assert result.all(), "result contains an inequality"

witness [ 1 14  1  2  3  6]


In [7]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = GF(2)
beta = GF(3)
gamma = GF(4)
delta = GF(5)
tau = GF(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")


print("Prover picks random r, s")
r = GF(12)
s = GF(13)

print(f"r = {r}")
print(f"s = {s}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20
Prover picks random r, s
r = 12
s = 13


In [8]:

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def poly_finite_field_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = galois.lagrange_poly(index, row)
        coef = poly.coefficients()[::-1]
        if len(coef) < matrics.shape[1]:
            coef = np.append(coef, np.zeros(matrics.shape[1] - len(coef), dtype=int))
        res.append(coef)
    return GF(res)

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=GF), galois.Poly(p2, field=GF)

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

def to_poly(matrix):
    poly_list = []
    for i in range(0, matrix.shape[0]):
        poly_list.append( galois.Poly(matrix[i][::-1]) )
    return poly_list

def evaluate_poly(poly, trusted_points):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"


    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    return evaluation

def evaluate_poly_list(poly_list, point):
    results = []
    for poly in poly_list:
        results.append(poly(point))
    return results



class ProverKey:
    def __init__(self, tau_G1, tau_G2, alpha_G1, beta_G1, beta_G2, delta_G1, delta_G2, K_delta_G1, target_G1):
        self.tau_G1 = tau_G1
        self.tau_G2 = tau_G2
        self.alpha_G1 = alpha_G1
        self.beta_G1 = beta_G1
        self.beta_G2 = beta_G2
        self.delta_G1 = delta_G1
        self.delta_G2 = delta_G2
        self.K_delta_G1 = K_delta_G1
        self.target_G1 = target_G1

    def __repr__(self):
        s = f"""
----- Prover Key -----
[τ]G1 = {[normalize(point) for point in self.tau_G1]}
[τ]G2 = {[normalize(point) for point in self.tau_G2]}
[α]G1 = {normalize(self.alpha_G1)}
[β]G1 = {normalize(self.beta_G1)}
[β]G2 = {normalize(self.beta_G2)}
[δ]G1 = {normalize(self.delta_G1)}
[δ]G2 = {normalize(self.delta_G2)}
[K/δ]G1 = {[normalize(point) for point in self.K_delta_G1]}
[τT(τ)/δ]G1 = {[normalize(point) for point in self.target_G1]}
        """
        return s
        
    def normalize(self):
        return Proof(normalize(self.A), normalize(self.B), normalize(self.C))


class VerifierKey:
    def __init__(self, alpha_G1, beta_G2, gamma_G2, delta_G2, K_gamma_G1):
        self.alpha_G1 = alpha_G1
        self.beta_G2 = beta_G2
        self.gamma_G2 = gamma_G2
        self.delta_G2 = delta_G2
        self.K_gamma_G1 = K_gamma_G1

    def __repr__(self):
        n_vk = self.normalize()
        s = f"""
----- Verifier Key -----
[α]G1 = {n_vk.alpha_G1}
[β]G2 = {n_vk.beta_G2}
[γ]G2 = {n_vk.gamma_G2}
[δ]G2 = {n_vk.delta_G2}
[K/γ]G1 = {n_vk.K_gamma_G1}
        """
        return s

    def normalize(self):
        return VerifierKey(normalize(self.alpha_G1), normalize(self.beta_G2), normalize(self.gamma_G2), normalize(self.delta_G2), [normalize(point) for point in self.K_gamma_G1])



def setup_keys(s, L, R, O):
    
    x = GF(np.arange(1, len(L) + 1))

    print(f"len(L): \n {len(L)}")

    L_galois = GF(L)
    R_galois = GF(R)
    O_galois = GF(O)

    resL = poly_finite_field_interpolate(x, np.transpose(L_galois))
    resR = poly_finite_field_interpolate(x, np.transpose(R_galois))
    resO = poly_finite_field_interpolate(x, np.transpose(O_galois))

    print(f"interporated L: \n {resL}")
    print(f"interporated R: \n {resR}")
    print(f"interporated O: \n {resO}")

    # print(f"resL.shape: \n {resL.shape}")
    # print(f"resO.shape: \n {resO.shape}")

    # resL.shape[0]

    beta_matrix = np.full( np.shape(resL) , beta, dtype=int)
    alpha_matrix = np.full( np.shape(resR), alpha, dtype=int)
    
    print(f"np.shape(resL): \n {np.shape(resL)}")
    print(f"beta_matrix: \n {beta_matrix}")

    beta_L = beta_matrix * resL
    alpha_R = alpha_matrix * resR
    K = beta_L + alpha_R + resO

    print(f"K: \n {K}")
    K_poly = to_poly(K)
    print_poly("K_poly", K_poly)

    print("K evaluations:")
    K_eval = evaluate_poly_list(K_poly, tau)
    print([int(k) for k in K_eval])

    U = inner_product_polynomials_with_witness(s, resL)
    V = inner_product_polynomials_with_witness(s, resR)
    W = inner_product_polynomials_with_witness(s, resO)

    # U_poly = galois.Poly((s @ L_galois)[::-1])
    # V_poly = galois.Poly((s @ R_galois)[::-1])
    # W_poly = galois.Poly((s @ O_galois)[::-1])

    U_poly = galois.Poly(U[::-1])
    V_poly = galois.Poly(V[::-1])
    W_poly = galois.Poly(W[::-1])

    print(f"L shape: {L.shape}")
    print(f"L : {L}")

    print(f"resL shape: {resL.shape}")
    print(f"resL : {resL}")

    T_poly = galois.Poly([1, P-1], field=GF)
    for i in range(2, resL.shape[1] + 1):
        print(f"T X- : {i}")
        T_poly *= galois.Poly([1, P-i], field=GF)

    print("\nT = ", T_poly)

    H_poly = (U_poly * V_poly - W_poly) // T_poly
    print("\nH = ",H_poly)
    rem = (U_poly * V_poly - W_poly) % T_poly
    print("\nrem = ", rem)
    assert rem == 0

    print("\nh(x)t(x = ", H_poly*T_poly)
    T_tau = T_poly(tau)
    print(f"\nT(τ) = {T_tau}")

    # u = U(tau)
    # v = V(tau)
    # _w = W(tau)
    # ht = H(tau)*T_tau

    # assert u * v == _w + ht

    # U1, U2 = split_poly(U)
    # V1, V2 = split_poly(V)
    # W1, W2 = split_poly(W)

    # w1 = W1(tau)
    # w2 = W2(tau)

    # u1 = U1(tau)
    # u2 = U2(tau)

    # v1 = V1(tau)
    # v2 = V2(tau)

    # c = (beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1
    # k = (beta * u1 + alpha * v1 + w1) * gamma**-1

    # a = u + alpha
    # b = v + beta

    # assert a * b == alpha * beta + k * gamma + c * delta

    # alpha_G1 = multiply(G1, int(alpha))
    # beta_G2 = multiply(G2, int(beta))
    # gamma_G2 = multiply(G2, int(gamma))
    # delta_G2 = multiply(G2, int(delta))

    
    pow_tauTtau_div_delta = [
        (tau**i * T_tau) / delta for i in range(0, T_poly.degree - 1)]
    target_G1 = [multiply(G1, int(pTd)) for pTd in pow_tauTtau_div_delta]

    K_gamma, K_delta = [k/gamma for k in K_eval[:2]
                        ], [k/delta for k in K_eval[2:]]

    tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T_poly.degree)]
    tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T_poly.degree)]
    alpha_G1 = multiply(G1, int(alpha))
    beta_G1 = multiply(G1, int(beta))
    beta_G2 = multiply(G2, int(beta))
    gamma_G2 = multiply(G2, int(gamma))
    delta_G1 = multiply(G1, int(delta))
    delta_G2 = multiply(G2, int(delta))
    K_gamma_G1 = [multiply(G1, int(k)) for k in K_gamma]
    K_delta_G1 = [multiply(G1, int(k)) for k in K_delta]

    print("Trusted setup:")
    print("-"*10)
    print(f"[α]G1 = {normalize(alpha_G1)}")
    print(f"[β]G2 = {normalize(beta_G2)}")
    print(f"[γ]G2 = {normalize(gamma_G2)}")
    print(f"[δ]G2 = {normalize(delta_G2)}")
    print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
    print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")
    print(f"[τT(τ)/δ]G1 = {[normalize(point) for point in target_G1]}")

    pk = ProverKey(tau_G1, tau_G2, alpha_G1, beta_G1, beta_G2, delta_G1,
                   delta_G2, K_delta_G1, target_G1)

    vk = VerifierKey(alpha_G1, beta_G2, gamma_G2, delta_G2, K_gamma_G1)

    return pk, vk



pk, vk  = setup_keys(w, L_F, R_F, O_F)


# U1, U2 = split_poly(U)
# V1, V2 = split_poly(V)
# W1, W2 = split_poly(W)

# w1 = W1(tau)
# w2 = W2(tau)

# u1 = U1(tau)
# u2 = U2(tau)

# v1 = V1(tau)
# v2 = V2(tau)

# a = u + alpha + r * delta
# b = v + beta + s * delta

# c = ((beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1) + s * a + r * b - r * s * delta
# k = (beta * u1 + alpha * v1 + w1) * gamma**-1

# assert a * b == alpha * beta + k * gamma + c * delta

len(L): 
 3
interporated L: 
 [[                                                                            0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0]
 [                                                                           10
  21888242871839275222246405745257275088548364400416034343698204186575808495608
                                                                              2]
 [                                                                            0
                                                                              0
                                                                              0]
 [2188

## Prover

In [9]:
class Proof:
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C

    def __repr__(self):
        n_proof = self.normalize()
        s = f"""
----- Proof -----
A = {n_proof.A}
B = {n_proof.B}
C = {n_proof.C}
        """
        return s

    def normalize(self):
        return Proof(normalize(self.A), normalize(self.B), normalize(self.C))
        

def generate_proof(pk: ProverKey, w_pub: [], w_priv: [], L_F, R_F, O_F):
    r = GF(12)
    s = GF(13)

    w = GF(np.concatenate((w_pub, w_priv)))

    L_galois = GF(L_F)
    R_galois = GF(R_F)
    O_galois = GF(O_F)

    x = GF(np.arange(1, len(L_F) + 1))

    resL = poly_finite_field_interpolate(x, np.transpose(L_galois))
    resR = poly_finite_field_interpolate(x, np.transpose(R_galois))
    resO = poly_finite_field_interpolate(x, np.transpose(O_galois))

    U = inner_product_polynomials_with_witness(w, resL)
    V = inner_product_polynomials_with_witness(w, resR)
    W = inner_product_polynomials_with_witness(w, resO)

    U_poly = galois.Poly(U[::-1])
    V_poly = galois.Poly(V[::-1])
    W_poly = galois.Poly(W[::-1])

    T_poly = galois.Poly([1, P-1], field=GF)
    for i in range(2, resL.shape[1] + 1):
        print(f"T X- : {i}")
        T_poly *= galois.Poly([1, P-i], field=GF)

    print("\nT = ", T_poly)

    H_poly = (U_poly * V_poly - W_poly) // T_poly
    print("\nH = ",H_poly)
    rem = (U_poly * V_poly - W_poly) % T_poly
    print("\nrem = ", rem)
    assert rem == 0


    # [K/δ*w]G1
    Kw_delta_G1_terms = [multiply(point, int(scaler))
                         for point, scaler in zip(pk.K_delta_G1, w_priv)]
    Kw_delta_G1 = Kw_delta_G1_terms[0]
    for i in range(1, len(Kw_delta_G1_terms)):
        Kw_delta_G1 = add(Kw_delta_G1, Kw_delta_G1_terms[i])

    r_delta_G1 = multiply(pk.delta_G1, int(r))
    s_delta_G1 = multiply(pk.delta_G1, int(s))
    s_delta_G2 = multiply(pk.delta_G2, int(s))

    A_G1 = evaluate_poly(U_poly, pk.tau_G1)
    A_G1 = add(A_G1, pk.alpha_G1)
    A_G1 = add(A_G1, r_delta_G1)

    B_G2 = evaluate_poly(V_poly, pk.tau_G2)
    B_G2 = add(B_G2, pk.beta_G2)
    B_G2 = add(B_G2, s_delta_G2)

    B_G1 = evaluate_poly(V_poly, pk.tau_G1)
    B_G1 = add(B_G1, pk.beta_G1)
    B_G1 = add(B_G1, s_delta_G1)

    As_G1 = multiply(A_G1, int(s))
    Br_G1 = multiply(B_G1, int(r))
    rs_delta_G1 = multiply(pk.delta_G1, int(-r*s))

    HT_G1 = evaluate_poly(H_poly, pk.target_G1)

    C_G1 = add(Kw_delta_G1, HT_G1)
    C_G1 = add(C_G1, As_G1)
    C_G1 = add(C_G1, Br_G1)
    C_G1 = add(C_G1, rs_delta_G1)


    return Proof(A_G1, B_G2, C_G1)

# pk, vk  = setup_keys(w, L_F, R_F, O_F)

w_pub = w[:2]
w_priv = w[2:]

generate_proof(pk,w_pub, w_priv, L_F, R_F, O_F)


T X- : 2
T X- : 3

T =  x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495611x^2 + 11x + 21888242871839275222246405745257275088548364400416034343698204186575808495611

H =  10944121435919637611123202872628637544274182200208017171849102093287904247805x + 21888242871839275222246405745257275088548364400416034343698204186575808495616

rem =  0



----- Proof -----
A = (11875194364178893954553492401677986714866872236848400464980908770614216104904, 1745931354231226481197267507063530252187087760877187346159283447433272042416)
B = ((6924602501121026249589381875774890071536197581515748953207543838373963201728, 17669910259781833670213145246136461402469144627494348846811202205232831152317), (6878733559265880876009410313708957190829192504316413095644358247575366168833, 14121526895642396450792006482533611011995333475901619442691494720354567354807))
C = (14301601145306053048655094763080599132319554666394446779504245502468178257858, 2660927896632112094165426621818690582616247844306821935940755268981089605284)
        